This notebook extract and calculate the VBS partitioning for the IGP.

In [1]:
#load modules
%run ../load_main.py
import salem
from WRFChemToolkit.analysis import aerosols_202 as aer, plots as pt, statistics as st, IGP as igp, utils as utl

WRFchem outputs data path is: dpath=/geos/d21/s1878599/IGP


## Load data

In [2]:
#Get IGP data for each season
aut_pth=dpath+'/outputs/autumn/wrfout*.nc'
win_pth=dpath+'/outputs/winter/wrfout*.nc'
mon_pth=dpath+'/outputs/monsoon/wrfout*.nc'
pmon_pth=dpath+'/outputs/premonsoon/wrfout*.nc'

shp_path =  '/exports/csce/datastore/geos/users/s1878599/phd_work/IGP/gadm_data/IGP/IGP.shp'
aIGP =igp.get_IGP(aut_pth, shp_path)
wIGP =igp.get_IGP(win_pth, shp_path)
mIGP =igp.get_IGP(mon_pth, shp_path)
pmIGP =igp.get_IGP(pmon_pth, shp_path)

In [3]:
#Get time and space avg aerosols for each IGP administration
def get_tsm_aerIGP(dict):
    tsaer_IGP={}
    for adm, val in dict.items():
        tsaer_IGP.update({adm : st.space_mean(st.time_mean(aer.get_aerosols(val),"Time"))})
    return tsaer_IGP

a= get_tsm_aerIGP(aIGP)
w= get_tsm_aerIGP(wIGP)
m= get_tsm_aerIGP(mIGP)
pm= get_tsm_aerIGP(pmIGP)

## Extract VBS

In [4]:
#Get only aerosols vbs for each size bin.
def get_vbs_aerbins(ds):    
    vbs= np.array([[(ds.asoaX_a01 + ds.bsoaX_a01).values[0],(ds.asoaX_a02  + ds.bsoaX_a02).values[0],(ds.asoaX_a03 + ds.bsoaX_a03).values[0],(ds.asoaX_a04 + ds.bsoaX_a04).values[0]],
                  [(ds.asoa1_a01 + ds.bsoa1_a01).values[0],(ds.asoa1_a02 + ds.bsoa1_a02).values[0],(ds.asoa1_a03 + ds.bsoa1_a03).values[0],(ds.asoa1_a04 + ds.bsoa1_a04).values[0]],
                  [(ds.asoa2_a01 + ds.bsoa2_a01).values[0],(ds.asoa2_a02 + ds.bsoa2_a02).values[0],(ds.asoa2_a03 + ds.bsoa2_a03).values[0],(ds.asoa2_a04 + ds.bsoa2_a04).values[0]],
                  [(ds.asoa3_a01 + ds.bsoa3_a01).values[0],(ds.asoa3_a02 + ds.bsoa3_a02).values[0],(ds.asoa3_a03 + ds.bsoa3_a03).values[0],(ds.asoa3_a04 + ds.bsoa3_a04).values[0]],
                  [(ds.asoa4_a01 + ds.bsoa4_a01).values[0],(ds.asoa4_a02 + ds.bsoa4_a02).values[0],(ds.asoa4_a03 + ds.bsoa4_a03).values[0],(ds.asoa4_a04 + ds.bsoa4_a04).values[0]]
                 ])
    return vbs   

In [5]:
#Get only aerosols and gas partitioning. Matrix 2x5
def get_vbs(ds):    
    cond_vap=[
        'cvasoaX','cvasoa1','cvasoa2','cvasoa3','cvasoa4',
        'cvbsoaX','cvbsoa1','cvbsoa2','cvbsoa3','cvbsoa4'
        ]
        
    aer = [
        'asoaX','asoa1','asoa2','asoa3','asoa4',
        'bsoaX','bsoa1','bsoa2','bsoa3','bsoa4'
        ]
        
    for v in aer:
        #sum each aer bin
        ds[v] = ds[v +'_a01'] + ds[v +'_a02'] +ds[v +'_a03']+ ds[v +'_a04']

    vbs= np.array([[(ds.asoaX + ds.bsoaX).values[0],(ds.asoa1 + ds.bsoa1).values[0],(ds.asoa2 + ds.bsoa2).values[0],(ds.asoa3 + ds.bsoa3).values[0],(ds.asoa4 + ds.bsoa4).values[0]],
                  [(ds.cvasoaX + ds.cvbsoaX).values[0],(ds.cvasoa1 + ds.cvbsoa1).values[0],(ds.cvasoa2 + ds.cvbsoa2).values[0],(ds.cvasoa3 + ds.cvbsoa3).values[0],(ds.cvasoa4 + ds.cvbsoa4).values[0]]
                 ])
    return vbs   

In [6]:
#extract VBS for each season and subregion
pmu_vbs = get_vbs(pm['U_IGP'])
pmm_vbs = get_vbs(pm['M_IGP'])
pml_vbs = get_vbs(pm['L_IGP'])

mu_vbs = get_vbs(m['U_IGP'])
mm_vbs = get_vbs(m['M_IGP'])
ml_vbs = get_vbs(m['L_IGP'])

au_vbs = get_vbs(a['U_IGP'])
am_vbs = get_vbs(a['M_IGP'])
al_vbs = get_vbs(a['L_IGP'])

wu_vbs = get_vbs(w['U_IGP'])
wm_vbs = get_vbs(w['M_IGP'])
wl_vbs = get_vbs(w['L_IGP'])

## Plot VBS

In [23]:
#DEFINE PLOT FUNCTION AND PLOT
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.orca.config.use_xvfb = True

def plot_vbs(vbs,row,col,letter,x,y):
    fig.add_trace(go.Bar(name='aerosol', x=bins,  y=[ vbs[0,0], 0, 0, 0, vbs[0,1],vbs[0,2],vbs[0,3],vbs[0,4]], marker_color="rgb(60,179,113)"),row=row,col=col)
    fig.add_trace(go.Bar(name='gas', x=bins,  y=[vbs[1,0],0,0,0,vbs[1,1],vbs[1,2],vbs[1,3],vbs[1,4]], marker_color="rgb(255,255,102)"),row=row,col=col)
    fig.add_annotation(
        x=x, #0.74,
        y=y, #0.98,
        xref="paper",
        yref="paper",
        text=letter,
        showarrow=False,
        font=dict(
            size=20,
            color="black"
        ),
        align="left",
        )


bins=['-4','-3','-2','-1', '0', '1','2', "3"]
fig = make_subplots(rows=4, cols=3,shared_yaxes=True,shared_xaxes=True,
                     horizontal_spacing = 0.03,vertical_spacing = 0.04)
col1_x=0.265
col2_x=0.630
col3_x=0.990

row1_y=0.99
row2_y=0.73
row3_y=0.45
row4_y=0.17

for i in fig['layout']['annotations']:
    i['font'] = dict(size=1)
plot_vbs(pmu_vbs,1,1,"(a)",col1_x,row1_y)
plot_vbs(pmm_vbs,1,2,"(b)",col2_x,row1_y)
plot_vbs(pml_vbs,1,3,"(c)",col3_x,row1_y)
plot_vbs(mu_vbs,2,1,"(d)",col1_x,row2_y)
plot_vbs(mm_vbs,2,2,"(e)",col2_x,row2_y)
plot_vbs(ml_vbs,2,3,"(f)",col3_x,row2_y)
plot_vbs(au_vbs,3,1,"(g)",col1_x,row3_y)
plot_vbs(am_vbs,3,2,"(h)",col2_x,row3_y)
plot_vbs(al_vbs,3,3,"(i)",col3_x,row3_y)
plot_vbs(wu_vbs,4,1,"(j)",col1_x,row4_y)
plot_vbs(wm_vbs,4,2,"(k)",col2_x,row4_y)
plot_vbs(wl_vbs,4,3,"(l)",col3_x,row4_y)


# Change the bar mode
fig.update_layout(barmode='stack')

fig.update_traces(showlegend=False)
fig.update_traces(showlegend=True, row=4, col=2)

fig.add_annotation(dict(text="$Organic \ Mass \ (\mu g \ m^{-3})$", x=-0.07, y=0.5,  xref="paper", yref="paper",font_size=100, showarrow=False,textangle=-90))
fig.add_annotation(dict(text="$log_{10}(C^*)$", x=0.5, y=-0.06,  xref="paper", yref="paper",font_size=100, showarrow=False))



fig.add_annotation(dict(text='UPPER', x=0.12, y=1.05, xref="paper", yref="paper",font_size=20, showarrow=False))
fig.add_annotation(dict(text='MIDDLE', x=0.5, y=1.05,  xref="paper", yref="paper",font_size=20, showarrow=False))
fig.add_annotation(dict(text='LOWER', x=0.9, y=1.05,  xref="paper", yref="paper",font_size=20, showarrow=False))


fig.add_annotation(dict(text='PRE-MONSOON', x=1.05, y=0.985,  xref="paper", yref="paper",font_size=20, showarrow=False,textangle=+90))
fig.add_annotation(dict(text='MONSOON', x=1.05, y=0.7,  xref="paper", yref="paper",font_size=20, showarrow=False,textangle=+90))
fig.add_annotation(dict(text='POST-MONSOON', x=1.05, y=0.27,  xref="paper", yref="paper",font_size=20, showarrow=False,textangle=+90))
fig.add_annotation(dict(text='WINTER', x=1.05, y=0.07,  xref="paper", yref="paper",font_size=20, showarrow=False,textangle=+90))

fig.update_layout(legend=dict(
    orientation="h",
    x=0.4,
    font=dict(
        #amily="Courier New, monospace",
        size=23
        #olor="#7f7f7f"
    )),font_size=20)
fig.update_yaxes(range=[0,15])
fig.update_layout(
    autosize=False,
    width=1000,
    height=1000)
fig.show()
fig.write_image('fig08.pdf')